In [ ]:
import h5py
import matplotlib
import pickle

import numpy as np
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc

from matplotlib import animation
from IPython.display import Video
from rich import print as print
from scipy.interpolate import griddata

import skies

%config InlineBackend.figure_format = "retina"

In [ ]:
# TODO: RBF Interpolation
# TODO: Spatial gradients in time
# TODO: Point at target folder
# TODO: Integrate into standard SKIES run
# TODO: Add earthquake events plot to bottom of figure
# TODO: Read only the data for the neccessary time steps at once to reduce memory footprint
# TODO: Contour plot for cumulative coseismic
# TODO: Countour plot for most recent event
# TODO: UUID instead of datetime for SKIES?
# TODO: Tricontourf: https://matplotlib.org/stable/gallery/images_contours_and_fields/tricontour_demo.html
# DONE: Compress hdf writing
# DONE: Label contour colorbar
# DONE: Plots of spatial probability
# TRY: https://stackoverflow.com/questions/42386372/increase-the-speed-of-redrawing-contour-plot-in-matplotlib

# Read in time series, mesh geometry, and mesh evolution

In [ ]:
params_file_name = "params.json"
time_series_file_name = "time_series.pickle"
meshes_parameters_file_name = "nankai_mesh_parameters.json"
hdf_filename = "2024_03_22_18_23_31.hdf"
hdf_filename = "2024_03_29_23_04_24.hdf"
hdf_filename = "2024_03_30_11_00_37.hdf"
hdf_filename = "2024_03_30_14_37_28.hdf"
hdf_filename = "2024_03_30_21_10_45.hdf"
hdf_filename = "2024_03_30_23_02_56.hdf"

# Read params
params = skies.get_params(params_file_name)

# Read time series
with open(time_series_file_name, "rb") as f:
    time_series = pickle.load(f)

print(
    f"Start time = {np.min(time_series.real_time)} years, End time = {np.max(time_series.real_time)} years"
)
print(
    f"Time steps = {len(time_series.real_time)}, Time step = {np.diff(time_series.real_time)[0]:0.4f} years = {np.diff(time_series.real_time)[0]*365:0.4f} days "
)


# Read mesh information
meshes = skies.read_meshes(meshes_parameters_file_name)

# Read hdf mesh evolution information
with h5py.File(hdf_filename, "r") as f:
    keys = list(f.keys())
    print(keys)
    for key in keys:
        print(f"{key}: {f[key][()].shape}")

    # Get cumulative slip
    cumulative_slip = f[keys[0]][()]

    # Get geometric moment
    geometric_moment = f[keys[1]][()]

    # Get geometric moment
    loading_rate = f[keys[2]][()]

    # Get location probability
    location_probability = f[keys[3]][()]

location_probability[np.isnan(location_probability)] = 0

# Constants

In [ ]:
frames = len(time_series.real_time)
frames = 100

In [ ]:
def plot_meshes(mesh, fill_value, ax, cmap_string):
    x_coords = mesh.meshio_object.points[:, 0]
    y_coords = mesh.meshio_object.points[:, 1]
    vertex_array = np.asarray(mesh.verts)

    if not ax:
        ax = plt.gca()
    xy = np.c_[x_coords, y_coords]
    verts = xy[vertex_array]
    pc = matplotlib.collections.PolyCollection(
        verts,
        edgecolor="None",
        cmap=cmap_string,
        linewidth=0.0,
        alpha=1.0,
    )
    pc.set_array(fill_value)
    ax.add_collection(pc)
    ax.autoscale()
    plt.gca().set_aspect("equal")
    return pc

# Animate cumulative coseismic slip

In [ ]:
fig, ax = plt.subplots()
to_plot = cumulative_slip[0, :]
pc = plot_meshes(meshes[0], to_plot, plt.gca(), cmc.hawaii_r)
plt.colorbar(pc, fraction=0.026, pad=0.04, label="slip (m)")
pc.set_clim(0, 1e1)
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k", linewidth=0.25)
plt.gca().set_facecolor("gainsboro")
plt.xlabel("longitude")
plt.ylabel("latitude")


def animate(i):
    pc.set_array(cumulative_slip[i, :])
    pc.set_clim(0, np.max(cumulative_slip[i, :]))
    plt.title(f"time step = {i}, real time = {time_series.real_time[i]:0.3f} (years)")


anim = animation.FuncAnimation(fig, animate, interval=1, frames=frames)
anim.save(filename="cumulative_slip.mp4", writer="ffmpeg", dpi=100)
plt.close("all")
Video("cumulative_slip.mp4", width=500, height=500)

# Animate geometric moment evolution (positive only)

In [ ]:
fig, ax = plt.subplots()
to_plot = geometric_moment[0, :] / meshes[0].areas
pc = plot_meshes(meshes[0], to_plot, plt.gca(), cmc.hawaii_r)
plt.colorbar(pc, fraction=0.026, pad=0.04, label="slip (m)")
# pc.set_clim(0, 1e1)
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k", linewidth=0.25)
plt.gca().set_facecolor("gainsboro")
plt.xlabel("longitude")
plt.ylabel("latitude")


def animate(i):
    to_plot = geometric_moment[i, :] / meshes[0].areas
    to_plot[to_plot < 0.0] = 0.0
    pc.set_array(to_plot)
    pc.set_clim(np.min(to_plot), np.max(to_plot))
    plt.title(f"time step = {i}, real time = {time_series.real_time[i]:0.3f} (years)")


anim = animation.FuncAnimation(fig, animate, interval=1, frames=frames)
anim.save(filename="geometric_moment_positive_only.mp4", writer="ffmpeg", dpi=100)
plt.close("all")
Video("geometric_moment_positive_only.mp4", width=500, height=500)

# Animate geometric moment evolution

In [ ]:
fig, ax = plt.subplots()
to_plot = geometric_moment[0, :] / meshes[0].areas
pc = plot_meshes(meshes[0], to_plot, plt.gca(), cmc.hawaii_r)
plt.colorbar(pc, fraction=0.026, pad=0.04, label="slip (m)")
# pc.set_clim(0, 1e1)
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k", linewidth=0.25)
plt.gca().set_facecolor("gainsboro")
plt.xlabel("longitude")
plt.ylabel("latitude")


def animate(i):
    to_plot = geometric_moment[i, :] / meshes[0].areas
    pc.set_array(to_plot)
    pc.set_clim(np.min(to_plot), np.max(to_plot))
    plt.title(f"time step = {i}, real time = {time_series.real_time[i]:0.3f} (years)")


anim = animation.FuncAnimation(fig, animate, interval=1, frames=frames)
anim.save(filename="geometric_moment.mp4", writer="ffmpeg", dpi=100)
plt.close("all")
Video("geometric_moment.mp4", width=500, height=500)

# Interpolated contour plot for geometric moment

In [ ]:
fig, ax = plt.subplots()
fill_value = geometric_moment[0, :] / meshes[0].areas
x_vec = np.linspace(params.min_longitude, params.max_longitude, params.n_grid_longitude)
y_vec = np.linspace(params.min_latitude, params.max_latitude, params.n_grid_latitude)
x_mat, y_mat = np.meshgrid(x_vec, y_vec)
centroids_lon = meshes[0].centroids[:, 0]
centroids_lat = meshes[0].centroids[:, 1]
fill_value_mat = griddata(
    (centroids_lon, centroids_lat), fill_value, (x_mat, y_mat), method="cubic"
)
# Set values outside of mesh polygon to nan so they don't plot
inpolygon_vals = skies.inpolygon(
    x_mat, y_mat, meshes[0].x_perimeter, meshes[0].y_perimeter
)
inpolygon_vals = np.reshape(
    inpolygon_vals, (params.n_grid_longitude, params.n_grid_latitude)
)
fill_value_mat[~inpolygon_vals] = np.nan
cmap = cmc.hawaii_r
# levels = np.linspace(-5e9, 5e9, 11)
# levels = 11

levels = np.linspace(
    np.min(geometric_moment[:, :] / meshes[0].areas),
    np.max(geometric_moment[:, :] / meshes[0].areas),
    50,
)

chf = plt.contourf(
    x_mat, y_mat, fill_value_mat, cmap=cmap, levels=levels, extend="both"
)
plt.colorbar(chf, fraction=0.026, pad=0.04, label="slip (m)")
ch = plt.contour(
    x_mat,
    y_mat,
    fill_value_mat,
    colors="k",
    linestyles="solid",
    linewidths=0.25,
    levels=levels,
)
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k", linewidth=1.0)
plt.gca().set_aspect("equal", adjustable="box")
plt.gca().set_facecolor("gainsboro")
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k")


def animate(i):
    global chf
    global ch
    # Remove filled contours
    for c in chf.collections:
        c.remove()
    # Remove line contours
    for c in ch.collections:
        c.remove()

    to_plot = geometric_moment[i, :] / meshes[0].areas
    fill_value_mat = griddata(
        (centroids_lon, centroids_lat), to_plot, (x_mat, y_mat), method="cubic"
    )
    inpolygon_vals = skies.inpolygon(
        x_mat, y_mat, meshes[0].x_perimeter, meshes[0].y_perimeter
    )
    inpolygon_vals = np.reshape(
        inpolygon_vals, (params.n_grid_longitude, params.n_grid_latitude)
    )
    fill_value_mat[~inpolygon_vals] = np.nan
    chf = plt.contourf(
        x_mat, y_mat, fill_value_mat, cmap=cmap, levels=levels, extend="both"
    )
    ch = plt.contour(
        x_mat,
        y_mat,
        fill_value_mat,
        colors="k",
        linestyles="solid",
        linewidths=0.25,
        levels=levels,
    )
    plt.title(f"time step = {i}, real time = {time_series.real_time[i]:0.3f} (years)")


del anim
anim = animation.FuncAnimation(fig, animate, interval=1, frames=frames)
anim.save(filename="geometric_moment_contour.mp4", writer="ffmpeg", dpi=100)
plt.close("all")
Video("geometric_moment_contour.mp4", width=500, height=500)

# Interpolate contour plot for location probabality

In [ ]:
%%prun
frames = 100

fig, ax = plt.subplots()
fill_value = location_probability[0, :]
x_vec = np.linspace(params.min_longitude, params.max_longitude, params.n_grid_longitude)
y_vec = np.linspace(params.min_latitude, params.max_latitude, params.n_grid_latitude)
x_mat, y_mat = np.meshgrid(x_vec, y_vec)
centroids_lon = meshes[0].centroids[:, 0]
centroids_lat = meshes[0].centroids[:, 1]
fill_value_mat = griddata(
    (centroids_lon, centroids_lat), fill_value, (x_mat, y_mat), method="cubic"
)
# Set values outside of mesh polygon to nan so they don't plot
inpolygon_vals = skies.inpolygon(
    x_mat, y_mat, meshes[0].x_perimeter, meshes[0].y_perimeter
)
inpolygon_vals = np.reshape(
    inpolygon_vals, (params.n_grid_longitude, params.n_grid_latitude)
)
fill_value_mat[~inpolygon_vals] = np.nan
cmap = cmc.hawaii_r
levels = np.linspace(
    np.min(location_probability),
    np.max(location_probability),
    50,
)

chf = plt.contourf(
    x_mat, y_mat, fill_value_mat, cmap=cmap, levels=levels, extend="both"
)
plt.colorbar(chf, fraction=0.026, pad=0.04, label="probability")
ch = plt.contour(
    x_mat,
    y_mat,
    fill_value_mat,
    colors="k",
    linestyles="solid",
    linewidths=0.25,
    levels=levels,
)
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k", linewidth=1.0)
plt.gca().set_aspect("equal", adjustable="box")
plt.gca().set_facecolor("gainsboro")
plt.plot(meshes[0].x_perimeter, meshes[0].y_perimeter, "-k")


def animate(i):
    global chf
    global ch
    # Remove filled contours
    for c in chf.collections:
        c.remove()
    # Remove line contours
    for c in ch.collections:
        c.remove()

    to_plot = location_probability[i, :]
    fill_value_mat = griddata(
        (centroids_lon, centroids_lat), to_plot, (x_mat, y_mat), method="cubic"
    )
    fill_value_mat[~inpolygon_vals] = np.nan
    chf = plt.contourf(
        x_mat, y_mat, fill_value_mat, cmap=cmap, levels=levels, extend="both"
    )
    ch = plt.contour(
        x_mat,
        y_mat,
        fill_value_mat,
        colors="k",
        linestyles="solid",
        linewidths=0.25,
        levels=levels,
    )
    plt.title(f"time step = {i}, real time = {time_series.real_time[i]:0.3f} (years)")


del anim
anim = animation.FuncAnimation(fig, animate, interval=1, frames=frames)
anim.save(filename="location_probability_contour.mp4", writer="ffmpeg", dpi=100)
# plt.close("all")
# Video("location_probability_contour.mp4", width=500, height=500)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Contour(
        z=[
            [10, 10.625, 12.5, 15.625, 20],
            [5.625, 6.25, 8.125, 11.25, 15.625],
            [2.5, 3.125, 5.0, 8.125, 12.5],
            [0.625, 1.25, 3.125, 6.25, 10.625],
            [0, 0.625, 2.5, 5.625, 10],
        ]
    )
)
fig.show()